In [2]:
import asyncio
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

def get_chrome_options():
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--disable-gpu") # Disable GPU acceleration
    chrome_options.add_argument("--window-size=1920x1080") # Set a default window size
    chrome_options.add_argument("--no-sandbox") # Bypass sandbox security
    chrome_options.add_argument("--user-agent='Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36'") # Set a user-agent
    return chrome_options

def get_temperature():
    driver = webdriver.Chrome(options=get_chrome_options())
    url = 'https://weather.com/weather/tenday/l/San+Diego+CA?canonicalCityId=cb5c473781cc06501376639dce8f0823a99187dcb42c79471a4303c076d66452'
    driver.get(url)
    temperature_element = driver.find_element(By.CSS_SELECTOR, 'span.DailyContent--temp--1s3a7')
    temperature = temperature_element.text
    driver.quit()
    return temperature

def get_stock_price():
    driver = webdriver.Chrome(options=get_chrome_options())
    url = 'https://finance.yahoo.com/quote/AAPL/'
    driver.get(url)
    stock_price_element = driver.find_element(By.CSS_SELECTOR, 'fin-streamer[data-symbol="AAPL"]')
    stock_price = stock_price_element.get_attribute('value')
    driver.quit()
    return stock_price


def get_current_time():
    driver = webdriver.Chrome(options=get_chrome_options())
    url = 'https://www.timeanddate.com/worldclock/usa/los-angeles'
    driver.get(url)
    time_element = driver.find_element(By.ID, 'ct')
    current_time = time_element.text
    driver.quit()
    return current_time

async def main():
    loop = asyncio.get_event_loop()
    temperature = await loop.run_in_executor(None, get_temperature)
    stock_price = await loop.run_in_executor(None, get_stock_price)
    print("Temperature:", temperature)
    print("Apple Stock Price:", stock_price)



async def run_async():
    while True: # Run the loop indefinitely
        loop = asyncio.get_event_loop()
        temperature = await loop.run_in_executor(None, get_temperature)
        stock_price = await loop.run_in_executor(None, get_stock_price)
        current_time = await loop.run_in_executor(None, get_current_time)
        print(f"Time: {current_time} | Temperature: {temperature} | Apple Stock Price: {stock_price}")
        
        await asyncio.sleep(60) # Wait for 60 seconds before the next iteration

In [3]:
# Run the async function
await run_async()

Time: 4:33:56 am | Temperature: 77° | Apple Stock Price: 177.79
Time: 4:35:09 am | Temperature: 77° | Apple Stock Price: 177.79
Time: 4:36:40 am | Temperature: 77° | Apple Stock Price: 177.79
Time: 4:37:54 am | Temperature: 77° | Apple Stock Price: 177.79
